In [12]:
import re
import math
from collections import Counter

putanja_korpus = r"C:\Users\nsabi\OneDrive\Desktop\SIS2 seminarski\korpus.txt"
putanja_dic = r"C:\Users\nsabi\OneDrive\Desktop\SIS2 seminarski\bs_BA.dic"

def ucitaj_i_ocisti(putanja):
    with open(putanja, 'r', encoding='utf-8') as f:
        tekst = f.read().lower()
    # Uzimaju se samo riječi (uključujući naša slova)
    return re.findall(r'[a-zčćžšđ]+', tekst)

# Učitavanje korpusa
sve_rijeci_korpus = ucitaj_i_ocisti(putanja_korpus)
brojac_unigrama = Counter(sve_rijeci_korpus)

# Učitavanje .dic rječnika
vokabular_dic = set()
try:
    with open(putanja_dic, 'r', encoding='utf-8') as f:
        for linija in f:
            rijec = linija.strip().split('/')[0].lower()
            if rijec.isalpha():
                vokabular_dic.add(rijec)
except Exception as e:
    print(f"Greška pri učitavanju rječnika: {e}")

# Unija riječi iz korpusa i rječnika
vokabular = set(brojac_unigrama.keys()).union(vokabular_dic)
print(f"Korpus učitan. Ukupno riječi: {len(sve_rijeci_korpus)}")
print(f"Vokabular spreman. Broj jedinstvenih riječi: {len(vokabular)}")
print(f"Prvih 5 najčešćih riječi: {brojac_unigrama.most_common(5)}")

Korpus učitan. Ukupno riječi: 4419
Vokabular spreman. Broj jedinstvenih riječi: 210546
Prvih 5 najčešćih riječi: [('i', 204), ('se', 129), ('u', 118), ('je', 81), ('da', 73)]


In [13]:
bigrami = {}

for i in range(len(sve_rijeci_korpus) - 1):
    w1 = sve_rijeci_korpus[i]
    w2 = sve_rijeci_korpus[i+1]
    
    if w1 not in bigrami:
        bigrami[w1] = Counter()
    bigrami[w1][w2] += 1

# P(c) - Unigram prior vjerovatnoća
ukupno_unigrama = sum(brojac_unigrama.values())
def get_unigram_prob(rijec):
    # Ako je riječ u korpusu, uzmi pravu frekvenciju
    if rijec in brojac_unigrama:
        return brojac_unigrama[rijec] / ukupno_unigrama
    # Ako je u rječniku ali nije u korpusu, daj joj šansu
    elif rijec in vokabular:
        return 0.000001 
    return 1e-10

# P(ct | ct-1) - Bigram vjerovatnoća prijelaza 
def get_transition_prob(prethodna, trenutna):
    if prethodna in bigrami and trenutna in bigrami[prethodna]:
        return (bigrami[prethodna][trenutna]) / sum(bigrami[prethodna].values())
    return get_unigram_prob(trenutna) * 0.1 

# PRIMJER
test_rijec = "ja"
if test_rijec in bigrami:
    print(f"Model je naučio da poslije riječi '{test_rijec}' najčešće idu:")
    for sljedeca, count in bigrami[test_rijec].most_common(3):
        vjerovatnoca = get_transition_prob(test_rijec, sljedeca)
        print(f" - {sljedeca}: {vjerovatnoca:.4f}")

Model je naučio da poslije riječi 'ja' najčešće idu:
 - idem: 0.3333
 - mislim: 0.2222
 - često: 0.1111


In [14]:
def get_base_char(char):
    izmjene = str.maketrans("čćžšđ", "cczsd")
    return char.translate(izmjene)

def damerau_levenshtein(s1, s2):
    d = {}
    len1, len2 = len(s1), len(s2)
    for i in range(-1, len1 + 1): d[(i, -1)] = i + 1
    for j in range(-1, len2 + 1): d[(-1, j)] = j + 1

    for i in range(len1):
        for j in range(len2):
            cost = 0 if s1[i] == s2[j] else 1
            d[(i, j)] = min(
                d[(i - 1, j)] + 1,      # Brisanje
                d[(i, j - 1)] + 1,      # Umetanje
                d[(i - 1, j - 1)] + cost # Zamjena
            )
            # Transpozicija susjednih znakova
            if i > 0 and j > 0 and s1[i] == s2[j-1] and s1[i-1] == s2[j]:
                d[(i, j)] = min(d[(i, j)], d[(i - 2, j - 2)] + cost)

    return d[(len1 - 1, len2 - 1)]

def get_emission_prob(unesena, kandidat, alfa=2.5):
    d = damerau_levenshtein(unesena, kandidat)
    vjerovatnoca = math.exp(-alfa * d)
    razlika_duzine = abs(len(unesena) - len(kandidat))
    if razlika_duzine > 1:
        vjerovatnoca *= 0.1 
    return vjerovatnoca

# PRIMJER
pogresna = "sistme"
ispravna = "sistem"
dist = damerau_levenshtein(pogresna, ispravna)
prob = get_emission_prob(pogresna, ispravna)

print(f"Udaljenost između '{pogresna}' i '{ispravna}' je: {dist}")
print(f"Vjerovatnoća greške P(w|c) za ovu udaljenost je: {prob:.4f}")

Udaljenost između 'sistme' i 'sistem' je: 1
Vjerovatnoća greške P(w|c) za ovu udaljenost je: 0.0821


In [15]:
def generisi_kandidate(rijec, vokabular, d_max=2):
    if rijec in vokabular:
        return [rijec]
    
    kandidati = []
    for ispravna_rijec in vokabular:
        if damerau_levenshtein(rijec, ispravna_rijec) <= d_max:
            kandidati.append(ispravna_rijec)
    return kandidati if kandidati else [rijec]

# PRIMJER
pogresna_rijec = "idm"
moguci_kandidati = generisi_kandidate(pogresna_rijec, vokabular, d_max=1)

print(f"Za unos '{pogresna_rijec}', pronađeni kandidati su:")
print(moguci_kandidati)

Za unos 'idm', pronađeni kandidati su:
['idim', 'ida', 'im', 'ibm', 'dim', 'idi', 'ide', 'idem', 'idu']


In [22]:
def viterbi_autocorrect(recenica_unos):
    opservacije = recenica_unos.lower().split()
    T = len(opservacije)
    viterbi_matrica = [{} for _ in range(T)]
    putanja = [{} for _ in range(T)]

    w1 = opservacije[0]
    kandidati_1 = generisi_kandidate(w1, vokabular)
    
    for c in kandidati_1:
        prior = get_unigram_prob(c)
        emisija = get_emission_prob(w1, c)
        viterbi_matrica[0][c] = math.log(prior) + math.log(emisija)
        putanja[0][c] = None

  
    for t in range(1, T):
        wt = opservacije[t]
        kandidati_t = generisi_kandidate(wt, vokabular)
        prethodni_kandidati = viterbi_matrica[t-1].keys()
        
        for c_t in kandidati_t:
            najbolja_vjerovatnoca = -float('inf')
            najbolji_prethodnik = None
            
            emisija = get_emission_prob(wt, c_t)
            
            for c_prev in prethodni_kandidati:
                prijelaz = get_transition_prob(c_prev, c_t)
                trenutni_score = viterbi_matrica[t-1][c_prev] + \
                 math.log(max(prijelaz, 1e-100)) + \
                 math.log(max(emisija, 1e-100))
                
                if trenutni_score > najbolja_vjerovatnoca:
                    najbolja_vjerovatnoca = trenutni_score
                    najbolji_prethodnik = c_prev
            
            viterbi_matrica[t][c_t] = najbolja_vjerovatnoca
            putanja[t][c_t] = najbolji_prethodnik

 
    zadnja_rijec = max(viterbi_matrica[T-1], key=viterbi_matrica[T-1].get)
    rekonstrukcija = []
    trenutna = zadnja_rijec
    for t in range(T-1, -1, -1):
        rekonstrukcija.append(trenutna)
        trenutna = putanja[t][trenutna]
        
    return " ".join(reversed(rekonstrukcija))

# PRIMJER
test_primjer = "Ja iskreno mislim da je ovaj predmet doosta zahgeevan" 
rezultat = viterbi_autocorrect(test_primjer)
print(f"Unos: {test_primjer}")
print(f"Viterbi ispravka: {rezultat}")

Unos: Ja iskreno mislim da je ovaj predmet doosta zahgeevan
Viterbi ispravka: ja iskreno mislim da je ovaj predmet dosta zahtjevan


In [17]:
# Test za analizu kandidata
test_w = "idm"
kandidati = generisi_kandidate(test_w, vokabular, d_max=1)
print(f"{'Kandidat':<12} | {'Log-Prior':<12} | {'Log-Emission':<12} | {'Ukupni Score':<12}")
print("-" * 55)
for c in kandidati:
    prior = math.log(get_unigram_prob(c))
    emission = math.log(get_emission_prob(test_w, c))
    score = prior + emission
    print(f"{c:<12} | {prior:<12.2f} | {emission:<12.2f} | {score:<12.2f}")

Kandidat     | Log-Prior    | Log-Emission | Ukupni Score
-------------------------------------------------------
idim         | -13.82       | -2.50        | -16.32      
ida          | -13.82       | -2.50        | -16.32      
im           | -8.39        | -2.50        | -10.89      
ibm          | -13.82       | -2.50        | -16.32      
dim          | -13.82       | -2.50        | -16.32      
idi          | -13.82       | -2.50        | -16.32      
ide          | -8.39        | -2.50        | -10.89      
idem         | -6.60        | -2.50        | -9.10       
idu          | -8.39        | -2.50        | -10.89      


In [27]:
def map_autocorrect(recenica_unos):
    rijeci = recenica_unos.lower().split()
    ispravljeno = []
    for w in rijeci:
        # MAP gleda samo unigram prior i emisiju (bez prethodne riječi)
        kandidati = generisi_kandidate(w, vokabular)
        najbolji = max(kandidati, key=lambda c: math.log(get_unigram_prob(c)) + math.log(get_emission_prob(w, c)))
        ispravljeno.append(najbolji)
    return " ".join(ispravljeno)

# Testni primjeri
test_primjeri = [
    "on ide u stolu",
    "razlicitih izbora",
    "digitalni ladi"
]

print(f"{'UNOS':<30} | {'MAP ':<25} | {'VITERBI ':<25}")
print("-" * 85)

for p in test_primjeri:
    res_map = map_autocorrect(p)
    res_viterbi = viterbi_autocorrect(p)
    print(f"{p:<30} | {res_map:<25} | {res_viterbi:<25}")

UNOS                           | MAP                       | VITERBI                  
-------------------------------------------------------------------------------------
on ide u stolu                 | on ide u stil             | on ide u školu           
razlicitih izbora              | različitih izvora         | različitih izvora        
digitalni ladi                 | digitalni ljudi           | digitalni alati          
